In [1]:
!pip install neuron
!pip install netpyne
!pip install inspyred
import matplotlib

     |████████████████████████████████| 12.6MB 16.3MB/s 
     |████████████████████████████████| 317kB 30.4MB/s 
     |████████████████████████████████| 92kB 8.2MB/s 


In [2]:
rm -r netpyne-course-2021

rm: cannot remove 'netpyne-course-2021': No such file or directory


In [3]:
!git clone --single-branch --branch evol https://github.com/suny-downstate-medical-center/netpyne-course-2021.git

Cloning into 'netpyne-course-2021'...
remote: Enumerating objects: 75, done.
remote: Counting objects: 100% (75/75), done.
remote: Compressing objects: 100% (60/60), done.
remote: Total 75 (delta 14), reused 67 (delta 12), pack-reused 0
Unpacking objects: 100% (75/75), done.


In [4]:
cd netpyne-course-2021

/content/netpyne-course-2021


In [5]:
ls

batchRun.py  cfg.py  init.py  netParams.py


In [6]:
from netpyne import specs, sim
%matplotlib inline
from netpyne import specs
from netpyne.batch import Batch

''' Example of evolutionary algorithm optimization of a network using NetPyNE
2 examples are provided: 'simple' and 'complex'
In 'simple', 3 parameters are optimized to match target firing rates in 2 populations
In 'complex', 6 parameters are optimized to match target firing rates in 6 populations

To run use: mpiexec -np [num_cores] nrniv -mpi batchRun.py
'''

def batchEvol(networkType):
	# parameters space to explore

	if networkType == 'simple':
		## simple net
		params = specs.ODict()
		params['prob'] = [0.01, 0.5]
		params['weight'] = [0.001, 0.1]
		params['delay'] = [1, 20]

		pops = {}
		pops['S'] = {'target': 5, 'width': 2, 'min': 2}
		pops['M'] = {'target': 15, 'width': 2, 'min': 0.2}

	elif networkType == 'complex':
		# complex net
		params = specs.ODict()
		params['probEall'] = [0.05, 0.2] # 0.1
		params['weightEall'] = [0.0025, 0.0075] #5.0
		params['probIE'] = [0.2, 0.6] #0.4
		params['weightIE'] = [0.0005, 0.002]
		params['probLengthConst'] = [100,200]
		params['stimWeight'] = [0.05, 0.2]

		pops = {}
		pops['E2'] = {'target': 5, 'width': 2, 'min': 1}
		pops['I2'] = {'target': 10, 'width': 5, 'min': 2}
		pops['E4'] = {'target': 30, 'width': 10, 'min': 1}
		pops['I4'] = {'target': 10, 'width': 3, 'min': 2}
		pops['E5'] = {'target': 40, 'width': 4, 'min': 1}
		pops['I5'] = {'target': 25, 'width': 5, 'min': 2}

	# fitness function
	fitnessFuncArgs = {}
	fitnessFuncArgs['pops'] = pops
	fitnessFuncArgs['maxFitness'] = 1000

	def fitnessFunc(simData, **kwargs):
		import numpy as np
		pops = kwargs['pops']
		maxFitness = kwargs['maxFitness']
		popFitness = [None for i in pops.items()]
		popFitness = [min(np.exp(  abs(v['target'] - simData['popRates'][k])  /  v['width']), maxFitness)
				if simData["popRates"][k]>v['min'] else maxFitness for k,v in pops.items()]
		fitness = np.mean(popFitness)
		popInfo = '; '.join(['%s rate=%.1f fit=%1.f'%(p,r,f) for p,r,f in zip(list(simData['popRates'].keys()), list(simData['popRates'].values()), popFitness)])
		print('  '+popInfo)
		return fitness

	# create Batch object with paramaters to modify, and specifying files to use
	b = Batch(params=params)

	# Set output folder, grid method (all param combinations), and run configuration
	b.batchLabel = 'simple'
	b.saveFolder = './'+b.batchLabel
	b.method = 'evol'
	b.runCfg = {
		'type': 'mpi_bulletin',#'hpc_slurm',
		'script': 'init.py',
		# options required only for hpc
		'mpiCommand': 'mpirun',
		'nodes': 1,
		'coresPerNode': 2,
		'allocation': 'default',
		'email': 'salvadordura@gmail.com',
		'reservation': None,
		'folder': '/home/salvadord/evol'
		#'custom': 'export LD_LIBRARY_PATH="$HOME/.openmpi/lib"' # only for conda users
	}
	b.evolCfg = {
		'evolAlgorithm': 'custom',
		'fitnessFunc': fitnessFunc, # fitness expression (should read simData)
		'fitnessFuncArgs': fitnessFuncArgs,
		'pop_size': 6,
		'num_elites': 1, # keep this number of parents for next generation if they are fitter than children
		'mutation_rate': 0.4,
		'crossover': 0.5,
		'maximize': False, # maximize fitness function?
		'max_generations': 4,
		'time_sleep': 5, # wait this time before checking again if sim is completed (for each generation)
		'maxiter_wait': 40, # max number of times to check if sim is completed (for each generation)
		'defaultFitness': 1000 # set fitness value in case simulation time is over
	}
	# Run batch simulations
	b.run()

# Main code
if __name__ == '__main__':
	batchEvol('simple')  # 'simple' or 'complex'



Saving batch to ./simple/simple_batch.json ... 
set prob=0.2244271249493722
set weight=0.013930141727217026
set delay=17.83365291678021
Saving simConfig to ./simple/gen_0/gen_0_cand_0_cfg.json ... 
--------------------------------------------------------------------------------
set prob=0.1960434257742902
set weight=0.08734609303201052
set delay=12.90449993227687
Saving simConfig to ./simple/gen_0/gen_0_cand_1_cfg.json ... 
--------------------------------------------------------------------------------
set prob=0.4277880653478038
set weight=0.07278293330863483
set delay=14.750730558574281
Saving simConfig to ./simple/gen_0/gen_0_cand_2_cfg.json ... 
--------------------------------------------------------------------------------
set prob=0.41773078005213476
set weight=0.05907113433018287
set delay=1.5125700770149608
Saving simConfig to ./simple/gen_0/gen_0_cand_3_cfg.json ... 
--------------------------------------------------------------------------------
set prob=0.18154745983852852

SystemExit: ignored